# Melting Polar Ice
Assignment 3
Ka Hei Pinky, Chow
U-number: 6899518

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

(i) Calculate the cartesian coordinates of the GPS receiver that recorded the observations, and the latitude/longitude of the location

In [ ]:
Define Function to convert ECEF coordinates to LLA

In [4]:
print("(i) Calculate the cartesian coordinates of the GPS receiver that recorded the observations, \
      and the latitude/longitude of the location")

# Define function to convert ECEF coordinates to LLA
def GPS_conversion(x, y, z):
    a = 6378137
    e = 8.1819190842622e-2

    asq = a**2
    esq = e**2

    b   = math.sqrt(asq * (1 - esq) )
    bsq = b**2

    ep  = math.sqrt((asq - bsq)/bsq)
    p   = math.sqrt(x**2 + y**2)
    th  = math.atan2(a * z, b * p)

    lon = math.atan2(y, x)
    lat = math.atan2( (z + math.pow(ep, 2) * b * math.pow(math.sin(th), 3)),\
                     (p - esq * a * math.pow(math.cos(th), 3)))
    N = a / (math.sqrt(1 - esq * math.pow(math.sin(lat), 2)))
    alt = p / math.cos(lat) - N

    lon = lon * 180 / math.pi
    lat = lat * 180 / math.pi
    return (round(lat, 6), round(lon, 6), round(alt, 6))

(i) Calculate the cartesian coordinates of the GPS receiver that recorded the observations,       and the latitude/longitude of the location


Input data and the constants 

In [ ]:
#import data from the file
data=np.loadtxt('pseudo.obs',skiprows=2)

sat_x=data[:,1]
sat_y=data[:,2]
sat_z=data[:,3]
sat_er=data[:,4]
pseudorange=data[:,5]
sigma=data[:,6]

#CONSTANTS
c = 299792458 #speed of light
n_obs = len(sat_x) 
n_params = 4 #the parameters are (x,y,z) and receiver error

x=1
y=1
z=1
re=1

In [ ]:
Parameters estimation

In [ ]:
#Least square method to estimate the parameters

A = np.zeros((n_obs,n_params))
b = np.zeros((n_obs))
xhat = np.zeros((n_params))

for iter in range(20):
    for i in range(n_obs): 
        A[i,0] = (x - sat_x[i])/math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)
        A[i,1] = (y - sat_y[i])/math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)
        A[i,2] = (z - sat_z[i])/math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)
        A[i,3] = c
        b[i] = pseudorange[i] - (math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)\
         + c*re - c*sat_er[i]*(10**-6)) #change microsec to sec

    At = np.transpose(A)
    AtA = np.dot(At,A)
    VCV = np.linalg.inv(AtA) 
    Atb = np.dot(At,b)
    xhat = np.dot(VCV,Atb)     
  
    x = x + xhat[0]
    y = y + xhat[1]
    z = z + xhat[2]
    re = re + xhat[3]
            
print("Final parameter estimates:   x= ", round(x,3))
print("Final parameter estimates:   y= ", round(y,3))
print("Final parameter estimates:   z= ", round(z,3)) 
print("Final parameter estimates:   receiver error= ", re) 
radius = 6378137 #meters

# Expected results:
Latitude,Longitude, Height (ellipsoidal) from ECEF

Latitude  : -69.74246   deg N
Longitude : 73.7112   deg E
Altitude    : 240.4   m


It is locateed in Amery ice shelf, Antarctica, close to Davis Station.

Print out GPS coordinates

In [3]:
lat, lon, alt = GPS_conversion(x, y, z)
print("Hence, the GPS coordinates of the location are estimated as:")
print("=================================")
print("Latitude:  ", lat)
print("Longitude:  ", lon)
print("Altitude:  ", alt)
print("=================================")  

NameError: name 'GPS_conversion' is not defined

(ii) The uncertainties of each of the estimated Cartesian coordinates and the receiver clock offset.

In [2]:
print("\n(ii) The uncertainties of each of the estimated Cartesian coordinates and\
      the receiver clock offset.")
print(VCV)

X_uncertainty = math.sqrt(VCV.item((0,0)))
Y_uncertainty = math.sqrt(VCV.item((1,1)))
Z_uncertainty = math.sqrt(VCV.item((2,2)))
Coff_uncertainty = math.sqrt(VCV.item((3,3)))

print("\nUncertainty of x = ", round(X_uncertainty, 3))
print("Uncertainty of y = ", round(Y_uncertainty, 3))
print("Uncertainty of z = ", round(Z_uncertainty, 3))
print("Uncertainty of clock offset = ", round(Coff_uncertainty, 10), "\n")


(ii) The uncertainties of each of the estimated Cartesian coordinates and      the receiver clock offset.
[[ 0.00015235  0.00007052 -0.00043812  0.        ]
 [ 0.00007052  0.00010729 -0.00032221  0.        ]
 [-0.00043812 -0.00032221  0.00246159 -0.        ]
 [ 0.          0.         -0.          0.        ]]

Uncertainty of x =  0.012
Uncertainty of y =  0.01
Uncertainty of z =  0.05
Uncertainty of clock offset =  1e-10 



(iii) Re-calculate the location taking into account the uncertainties of each pseudorange observation. 
By how much do the parameter estimates (and the uncertainties) change?

In [3]:
print("(iii) Re-calculate the location taking into account the uncertainties of each pseudorange \
      observation. By how much do the parameter estimates (and the uncertainties) change?")
#Weighted least square
W = np.zeros((11, 11))

for i in range(0, 11):
    np.set_printoptions(suppress=True)
    W[i, i] = 1/((sigma[i])**2)
print(W)
#print(np.array_str(W, precision=1, suppress_small=True))

xq=1
yq=1
zq=1
req=1

A = np.zeros((n_obs,n_params))
b = np.zeros((n_obs))
xhat = np.zeros((n_params))

for iter in range(20):
    for i in range(n_obs): 
        A[i,0] = (x - sat_x[i])/math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)
        A[i,1] = (y - sat_y[i])/math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)
        A[i,2] = (z - sat_z[i])/math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)
        A[i,3] = c
        b[i] = pseudorange[i] - (math.sqrt((sat_x[i] - x)**2 + (sat_y[i] - y)**2 + (sat_z[i] - z)**2)\
         + c*re - c*sat_er[i]*(10**-6)) #change microsec to sec

    At = np.transpose(A)
    AtW = np.dot(At,W)
    AtWA = np.dot(AtW,A)
    VCV = np.linalg.inv(AtWA) 
    AtWb = np.dot(AtW,b)
    xhat = np.dot(VCV,AtWb)
  
    x = x + xhat[0]
    y = y + xhat[1]
    z = z + xhat[2]
    re = re + xhat[3]

print("After taking into account the uncertainties of each pseudorange observation:")
print("New parameter estimates:   x= ", round(x,3))
print("New parameter estimates:   y= ", round(y,3))
print("New Parameter estimates:   z= ", round(z,3)) 
print("New Parameter estimates:   receiver error= ", np.around(re, 10))

(iii) Re-calculate the location taking into account the uncertainties of each pseudorange       observation. By how much do the parameter estimates (and the uncertainties) change?


NameError: name 'np' is not defined

In [ ]:
New GPS coordinates

In [ ]:
new_lat, new_lon, new_alt = GPS_conversion(x, y, z)
print("Hence, the new GPS coordinates of the location are estimated as:")
print("=================================")
print("adjusted_Latitude:  ", new_lat)
print("adjusted_Longitude:  ", new_lon)
print("adjusted_Altitude:  ", new_alt)
print("=================================")    

The new uncertainties of each of the estimated Cartesian coordinates and the receiver clock offset are:

In [1]:
print("The new uncertainties of each of the estimated Cartesian coordinates and the receiver clock offset are:")
print(VCV)

NEWX_uncertainty = math.sqrt(VCV.item((0,0)))
NEWY_uncertainty = math.sqrt(VCV.item((1,1)))
NEWZ_uncertainty = math.sqrt(VCV.item((2,2)))
NEWCoff_uncertainty = math.sqrt(VCV.item((3,3)))

print("\nUncertainty of x = ", round(NEWX_uncertainty, 3))
print("Uncertainty of y = ", round(NEWY_uncertainty, 3))
print("Uncertainty of z = ", round(NEWZ_uncertainty, 3))
print("Uncertainty of clock offset = ", round(NEWCoff_uncertainty, 10), "\n")

The new uncertainties of each of the estimated Cartesian coordinates and the receiver clock offset are:


NameError: name 'VCV' is not defined

By how much do the parameter estimates (and the uncertainties) change?

In [2]:
print("Changes of Latitude: ",(new_lat - lat))
print("Changes of Longitude: ",(new_lon - lon))
print("Changes of Altitude: ",(new_alt - alt))
print("Changes of X uncertainties: ",(NEWX_uncertainty - X_uncertainty))
print("Changes of Y uncertainties: ",(NEWY_uncertainty - Y_uncertainty))
print("Changes of Z uncertainties: ",(NEWZ_uncertainty - Z_uncertainty))
print("Changes of uncertainty of clock offset: ",(NEWCoff_uncertainty - Coff_uncertainty))

NameError: name 'new_lat' is not defined

# Expected Results of new GPS coordinates

adjusted_Latitude:   -69.742376
adjusted_Longitude:   73.711225
adjusted_Altitude:   204.048381

The location is Indian Ocean at the coast of Antarctica. 


Correlation Matrix

In [1]:
sigmaX = math.sqrt(VCV.item((0,0)))
sigmaY = math.sqrt(VCV.item((1,1)))
sigmaZ = math.sqrt(VCV.item((2,2)))

sigmaXY = VCV.item((0,1))
sigmaXZ = VCV.item((0,2))
sigmaYZ = VCV.item((2,1))

print("\n(iv) Calculate the correlations between each parameter")
print("Calculting the correlation between x and y: ")
print(round(sigmaXY/(sigmaX*sigmaY), 6))

print("Calculting the correlation between x and z: ")
print(round(sigmaXZ/(sigmaX*sigmaZ), 6))

print("Calculting the correlation between y and z: ")
print(round(sigmaYZ/(sigmaY*sigmaZ), 6), "\n")

print("VCV matrix")
print(VCV)
VCV_ = VCV.copy()
print("\nVCV matrix without receiver error")
print(VCV_[:-1,:-1])


corrl = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        corrl[i, j] = VCV_[:-1,:-1][i, j]/(np.sqrt(VCV_[:-1,:-1][i,i])*np.sqrt(VCV_[:-1,:-1][j,j]))

print("\nCorrelation Matrix")
print(corrl)

(i) Calculate the cartesian coordinates of the GPS receiver that recorded the observations,       and the latitude/longitude of the location
Final parameter estimates:   x=  621257.504
Final parameter estimates:   y=  2126077.905
Final parameter estimates:   z=  -5961378.288
Final parameter estimates:   receiver error=  2.0775795153040817e-07
Hence, the GPS coordinates of the location are estimated as:
Latitude:   -69.742461
Longitude:   73.711202
Altitude:   240.427605

(ii) The uncertainties of each of the estimated Cartesian coordinates and      the receiver clock offset.
[[ 2.84107178e-01 -1.97739043e-02  1.48122464e-01 -3.99645769e-10]
 [-1.97739043e-02  3.46484789e-01 -4.16448239e-01  7.84556807e-10]
 [ 1.48122464e-01 -4.16448239e-01  1.98012699e+00 -3.24054597e-09]
 [-3.99645769e-10  7.84556807e-10 -3.24054597e-09  6.45178296e-18]]

Uncertainty of x =  0.533
Uncertainty of y =  0.589
Uncertainty of z =  1.407
Uncertainty of clock offset =  0.0 

(iii) Re-calculate the location t

End of Assignment 3